# 使用future处理并发

In [ ]:
# 在I/O密集型应用中，如果代码写得正确，那么不管使用哪种并发策略（使用线程或asyncio包），吞吐量都比依序执行的代码高很多。

In [1]:
import os
import time
import sys

import requests
from concurrent import futures

In [2]:
from time import sleep, strftime

In [ ]:
# 依序下载的脚本
POP20_CC = 'CN IN US ID BR PK NG BD RU JP MX PH VN ET EG DE IR TR CD FR'.split()
BASE_URL = 'http://flupy.org/data/flags'
DEST_DIR = 'downloads/'

def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp .write(img)

def get_flag(cc):
    url = '{}/{cc}/{cc}.git'.format(BASE_URL, cc = cc.lower())
    resp = requests.get(url)
    return resp.content

def show(text):
    print (text, end = ' ')
    sys.stdout.flush()

def download_many(cc_list):
    for cc in sorted(cc_list):
        imag = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')
    return len(cc_list)

def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print (msg.format(count, elapsed))
    
#多线程下载脚本
MAX_WORKERS = 20
def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_one, sorted(cc_list))
    return len(list(res))

In [ ]:
# future 在哪？future是concurrent.futures 和 asyncio包的重要组件
def download_many(cc_list):
    cc_list = cc_list[:5]
    with futures.ThreadPoolExecutor(max_workers = 3) as executor:
        to_do = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            to_do.append(future)
            msg = 'Scheduled for {}: {}'
            prit (msg.format(cc, future))

        result = []
        # as_completed 返回的是future运行结束后产出future的迭代器，不会阻塞调用方线程
        for future in futures.as_completed(to_do):
            res = future.result()
            msg = '{} result: {!r}'
            print (msg.format(future, res))
            results.append(res)
    return len(results)

In [ ]:
# 目前测试的并发脚本都不能并行下载，使用concurrent.futures库实现的两个示例受GIL(Global Interpreter Lock，全局解释锁)的限制
# 两个疑问：
# 既然python线程受GIL的限制，任何时候都只允许运行一个线程，那么flags_threadpool.py脚本的下载速度怎么会提升5倍
# flags_asyncio.py脚本和flags.py脚本都在单个线程中运行，前者为啥快5倍

In [ ]:
# CPython解释器本身就不是线程安全的，GIL一次只允许使用一个线程执行Python字节码，一个Python进程通常不能同时使用多个CPU核心

In [ ]:
# 一个Python线程在等待网络响应时，阻塞型I/O函数会释放GIL，再运行一个线程。
# “Python 线程毫无作用” by David Beazley

In [ ]:
# ProcessPoolExecutor 类把工作分配给多个Python进程处理，因此需要做CPU密集型处理，使用这个模块能绕开GIL，利用所有可用的CPU核心
# 最佳线程数取决于做的是什么事，以及可以用的内存有多少。
# 最佳进程数一般不要超过CPU数量

In [7]:
def display(*args):
    print (strftime('[%H:%M:%S]'), end=' ')
    print (*args)

def loiter(n):
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}): done.'
    display(msg.format('\t'*n, n))
    return n * 10

def test():
    display('Script starting.')
    executor = futures.ThreadPoolExecutor(max_workers=3)
    results = executor.map(loiter, range(5))
    display('results:', results)
    display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('result {}: {}'.format(i, result))

In [8]:
test()

[14:22:10] Script starting.
[14:22:10] loiter(0): doing nothing for 0s...
[14:22:10] loiter(0): done.
[14:22:10] 	loiter(1): doing nothing for 1s...
[14:22:10] 		loiter(2): doing nothing for 2s...[14:22:10]
[14:22:10] results: <generator object Executor.map.<locals>.result_iterator at 0x110eb8f48>
[14:22:10] Waiting for individual results:
[14:22:10] result 0: 0
 			loiter(3): doing nothing for 3s...
[14:22:11] 	loiter(1): done.
[14:22:11] 				loiter(4): doing nothing for 4s...
[14:22:11] result 1: 10
[14:22:12] 		loiter(2): done.
[14:22:12] result 2: 20
[14:22:13] 			loiter(3): done.
[14:22:13] result 3: 30
[14:22:15] 				loiter(4): done.
[14:22:15] result 4: 40


In [9]:
# 更高级的工具 threading 和 multiprocessing模块
# 解决协作进程遇到的最大挑战: 在进程之间传递数据